<a href="https://colab.research.google.com/github/papoularubra/clienton/blob/main/ClientOn_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip -q install google-genai

In [12]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [13]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [10]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [21]:
!pip install -U googlemaps

In [25]:
import googlemaps

# Substitua 'SUA_CHAVE_DE_API' pela sua chave de API real
api_key = 'GOOGLE_PLACESgmaps = googlemaps.Client(key=api_key)

# Agora você pode usar o objeto 'gmaps' para interagir com a API
# Exemplo de uma chamada à API de geocodificação:
try:
    geocode_result = gmaps.geocode('Avenida Paulista, São Paulo')
    print(geocode_result)
except Exception as e:
    print(f"Ocorreu um erro ao chamar a API: {e}")

[{'address_components': [{'long_name': 'Avenida Paulista', 'short_name': 'Av. Paulista', 'types': ['route']}, {'long_name': 'São Paulo', 'short_name': 'São Paulo', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'São Paulo', 'short_name': 'SP', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Brazil', 'short_name': 'BR', 'types': ['country', 'political']}], 'formatted_address': 'Av. Paulista, São Paulo - SP, Brazil', 'geometry': {'bounds': {'northeast': {'lat': -23.5525567, 'lng': -46.64398449999999}, 'southwest': {'lat': -23.5721549, 'lng': -46.6657058}}, 'location': {'lat': -23.5613545, 'lng': -46.6564943}, 'location_type': 'GEOMETRIC_CENTER', 'viewport': {'northeast': {'lat': -23.5525567, 'lng': -46.64398449999999}, 'southwest': {'lat': -23.5721549, 'lng': -46.6657058}}}, 'place_id': 'ChIJFaMK2shZzpQRanfJ8jGUn9U', 'types': ['route']}]


In [14]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from googlemaps import places
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [15]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [16]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [39]:
##########################################
# -- Agente 1: Buscador de Informações -- #
##########################################

def agente_buscador(topico, data_de_hoje):

# Busca no Google Places
    places_result = gmaps.find_place(input=topico, input_type='textquery', fields=['name', 'formatted_address', 'rating', 'user_ratings_total', 'photos'])


    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa especializado em due diligence de empresas.
        Sua tarefa é realizar uma investigação abrangente sobre a reputação e histórico da empresa indicada em topico.
        Para isso, utilize a ferramenta de busca do Google (google_search) para:
        - Identificar informações básicas da empresa:
        CNPJ, data de fundação/tempo de existência, endereço, nicho de mercado principal e outras informações cadastrais relevantes.
        - Consultar a reputação online:
        Buscar e analisar avaliações e reclamações sobre a empresa em topico no site Reclame Aqui (reclameaqui.com.br).
        Explorar outras plataformas de avaliação de consumidores e reputação de empresas (se disponíveis e relevantes para o contexto brasileiro).
        Investigar comentários em redes sociais da empresa ou sobre a empresa em topico.
        - Buscar notícias sobre a empresa em topico, fotos e avaliações no API Places do Google.
        - Verificar pendências legais e regulatórias:
        Pesquisar por processos judiciais e menções da empresa em topico no Jusbrasil e sites jurídicos oficiais.
        Verificar a existência de reclamações ou processos administrativos contra a empresa em topico
        no Procon e outros órgãos reguladores relevantes para o nicho da empresa.
        Formato: Você vai realizar toda essa pesquisa, mas só vai mostrar as informações básicas e fundamentais neste passo, em formato tabulado.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Empresa: {topico}\nData de hoje: {data_de_hoje}\nPlaces Result: {places_result}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [30]:
################################################
# --- Agente 2: Avaliador de reputação --- #
################################################

def agente_avaliador(topico, lancamentos_buscados):
    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash", #não foi possível usar um modelo mais avançado, mas seria o ideal

        # Inserir as instruções do Agente Avaliador #################################################
        instruction="""
        Você é um orientador de consumidores especializado em due diligence de empresas.
        Com base em
        1 - nas informações levantadas pelo agente buscador, e/ou
        2 - na análise dos requisitos de segurança do link indicado em tópico, se for um link
        (Caso o usuário peça informações a partir de um link, realize as ações anteriores cabíveis e também:
        - Identifique se o link é seguro.
        - Identifique se o link é de propriedade da empresa sugerida.
        - Verifique os selos de segurança do site, a política de privacidade, certificado SSL e outros critérios de segurança
        que ajudem a evitar que o usuário caia em um golpe.),
        você deve:
        - Usar a ferramenta de busca do Google (google_search) para indicar se a empresa é confiável ou não e
        elaborar uma justificativa para a afirmação, de maneira sucinta, linguagem clara e acessível.
        - Se possível, categorize os tipos de reclamações ou avaliações
        (por exemplo, problemas com produto, atendimento, entrega) para fornecer uma visão mais detalhada da reputação.
        - Mencione a data dos principais comentários e categorize como "Elogio", "Reclamação" ou "Neutro".
        Destaque a informação desse agente em relação aos outros, pois é a mais importante.
        Destaque a conclusão sobre a empresa ou site ser confiável ou não colocando as expressões "É CONFIÁVEL" ou "NÃO É CONFIÁVEL"
        em negrito e caixa alta, esta informação deve estar na primeira linha do texto.
        Formato: As informações devem ser sucintas, com linguagem clara e acessível, apresentadas de  modo tabulado
        com os dados positivos em verde e os negativos em vermelho.
        """,
        description="Agente que avalia a confiabilidade da empresa",
        tools=[google_search]
    )

    entrada_do_agente_avaliador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    avaliacao_da_empresa = call_agent(avaliador, entrada_do_agente_avaliador)
    return avaliacao_da_empresa

In [19]:
######################################
# --- Agente 3: Consultor --- #
######################################
def agente_consultor(topico, avaliacao_da_empresa):
    consultor = Agent(
        name="agente_consultor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um consultor de consumidores, com foco em oferecer respostas claras e concisas para ajudá-los
            no consumo consciente e seguro diante da escolha de uma empresa.
            Com base nas informações dos lacamentos_buscados e avaliacao_da_empresa, você deve:
            - Considerar o tom geral das avaliações e reclamações encontradas pelo agente_buscador e agente_avaliador.
            - Indicar ao cliente se deve ou não negociar com a empresa ou comprar um produto dela.
            - Indicar para qual público e finalidade a empresa é indicada.
            - Se cabível, indicar aspectos de responsabilidade social, compliance, sustentabilidade e
            outras práticas que forem um diferencial da empresa.
            - Se a empresa for CONFIÁVEL: mencionar pontos positivos mais relevantes com base na recorrência do tema nas avaliações.
            - Se cabível, indicar concorrente para comparação, apenas se tiver qualidades compatíveis ou superiores às da empresa pesquisada.
            - Compare a reputação da empresa consultada com a média do setor, se houver dados disponíveis.
            - Considere que empresas sem informações cadastrais online, como CNPJ, não são confiáveis.
            Formato: Essa conclusão deve ser sucinta, com linguagem clara e acessível, ser tabulada, os dados positivos em verde e os negativos em vermelho.
            """,
        description="Agente consultor de negocios com foco no cliente"
    )
    entrada_do_agente_consultor = f"Tópico: {topico}\nPlano de post: {avaliacao_da_empresa}"
    # Executa o agente
    decisao_consumidor = call_agent(consultor, entrada_do_agente_consultor)
    return decisao_consumidor

In [40]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🧐 Iniciando o Sistema de Avaliação de Confiança de Empresas 🧐")

# --- Obter o Tópico do Usuário ---
topico = input("Digite o link do site ou o nome da empresa sobre a qual você deseja informações: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos saber mais sobre a empresa {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Informações sobre a empresa ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    avaliacao_da_empresa = agente_avaliador(topico, lancamentos_buscados)
    print("\n---❓Análise de confiança ---\n")
    display(to_markdown(avaliacao_da_empresa))
    print("--------------------------------------------------------------")

    decisao_consumidor = agente_consultor(topico, avaliacao_da_empresa)
    print("\n--- 👨‍⚖️ Sugestão de decisão para o consumidor ---\n")
    display(to_markdown(decisao_consumidor))
    print("--------------------------------------------------------------")



🧐 Iniciando o Sistema de Avaliação de Confiança de Empresas 🧐
Digite o link do site ou o nome da empresa sobre a qual você deseja informações: Padaria Palma de Ouro
Maravilha! Vamos saber mais sobre a empresa Padaria Palma de Ouro

--- 📝 Informações sobre a empresa ---



> Com base nas informações fornecidas e na data atual (17 de maio de 2025), realizei uma busca abrangente sobre a Padaria Palma de Ouro. Abaixo, apresento um resumo das informações encontradas:
> 
> | Categoria                  | Informação                                                                                                                                                                                                                                               |
> | -------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
> | Nome da Empresa            | Padaria Palma de Ouro                                                                                                                                                                                                                                   |
> | Endereço                   | R. Japurá, 11 - Bela Vista, São Paulo - SP, 01319-030, Brazilië                                                                                                                                                                                         |
> | Avaliação do Google Places | 4.4 estrelas (com base em 2927 avaliações)                                                                                                                                                                                                                 |
> | Reputação Online           | Será necessário realizar buscas no Reclame Aqui e outras plataformas de avaliação para determinar a reputação online da empresa.                                                                                                                          |
> | Pendências Legais          | Será necessário pesquisar por processos judiciais e administrativos no Jusbrasil, Procon e outros órgãos reguladores.                                                                                                                                    |
> | Outras Informações         | Informações adicionais, como CNPJ, data de fundação e nicho de mercado principal, serão buscadas para complementar a análise. A pesquisa incluirá também redes sociais e notícias para obter uma visão mais completa do histórico e reputação da empresa. |
> 
> Para dar seguimento à análise de *due diligence*, vou realizar as seguintes buscas no Google:
> 
> 
> Com base nos resultados da pesquisa, apresento as informações atualizadas e consolidadas sobre a Padaria Palma de Ouro:
> 
> **Informações Cadastrais:**
> 
> *   **Nome da Empresa:** Foram encontradas algumas empresas com o nome "Palma de Ouro", sendo necessário identificar qual corresponde à padaria em questão:
>     *   **PADARIA E CONFEITARIA PALMA DE OURO LTDA:** CNPJ 11385787000173 (Localizada em Salto, São Paulo)
>     *   **PAES E DOCES PALMA DE OURO DE SANTOS LTDA - EPP (PALMA DE OURO DE SANTOS):** CNPJ 27.808.353/0001-79 (Fundada em 24/05/2017, Ativa, Principal atividade: Padaria e confeitaria com predominância de revenda, Localizada em Santos, SP)
>     *   **PALMA DE OURO DISTRIBUIDORA DE ALIMENTOS LTDA - EPP:** CNPJ 33.564.663/0001-98 (Fundada em 08/05/2019, Ativa, Principal atividade: Comércio varejista de produtos alimentícios em geral, Localizada em São Paulo, SP)
>     *   **PANIFICADORA PALMA DE OURO DO CENTRO LTDA - EPP:** (Mencionada em lista de geradores cadastrados na Prefeitura de São Paulo)
> *   **Endereço:** R. Japurá, 11 - Bela Vista, São Paulo - SP, 01319-030 (Conforme Places Result)
> *   **Nicho de Mercado:** Padaria, Confeitaria, Restaurante (conforme informações do website e resultados da busca)
> *   **Tempo de Existência:** Depende de qual CNPJ corresponde à padaria do endereço informado. A pesquisa indica que a "Palma de Ouro" original foi inaugurada nos anos 60.
> 
> **Reputação Online:**
> 
> *   **Google Places:** Avaliação de 4.4 estrelas com base em 2927 avaliações.
> *   **Reclame Aqui:** Necessário verificar a presença e reputação da empresa nas plataformas Reclame Aqui.
> 
> **Pendências Legais e Regulatórias:**
> 
> *   **Processos Judiciais:** Necessário pesquisar processos judiciais associados aos CNPJs identificados.
> *   **Procon:** Necessário verificar a existência de reclamações ou processos administrativos no Procon.
> 
> **Informações Adicionais:**
> 
> *   A Padaria Palma de Ouro possui um website ([https://palmadeouro.com.br/](https://palmadeouro.com.br/)), onde oferece informações sobre cardápio, história e serviços.
> *   A pesquisa revelou um "Edital de Abertura de Procedimento Administrativo de Reconhecimento de Responsabilidade" (20/08/2024) envolvendo empresas com nomes similares, sendo necessário verificar se a Padaria Palma de Ouro está relacionada.
> 
> Para continuar a *due diligence*, os próximos passos incluem:
> 
> 1.  **Confirmar o CNPJ correto** da Padaria Palma de Ouro localizada no endereço R. Japurá, 11 - Bela Vista, São Paulo - SP.
> 2.  **Analisar a reputação da empresa no Reclame Aqui** e outras plataformas de avaliação, buscando por reclamações e avaliações de clientes.
> 3.  **Investigar a existência de processos judiciais e administrativos** (Procon, Vigilância Sanitária, etc.) associados ao CNPJ correto.
> 4.  **Verificar o "Edital de Abertura de Procedimento Administrativo"** para determinar se a empresa está envolvida em alguma questão legal ou regulatória.
> 


--------------------------------------------------------------

---❓Análise de confiança ---



> **NÃO É CONFIÁVEL**
> 
> Com base nas informações disponíveis até o momento, não é possível atestar a confiabilidade da Padaria Palma de Ouro sem concluir as etapas de investigação pendentes. A existência de homônimos com CNPJs distintos e um edital administrativo envolvendo nomes similares levantam questões que precisam ser esclarecidas.
> 
> Para determinar se a Padaria Palma de Ouro é confiável, é crucial seguir os próximos passos:
> 
> *   Confirmar o CNPJ correto da padaria localizada no endereço R. Japurá, 11 - Bela Vista, São Paulo - SP.
> 
> *   Analisar a reputação da empresa no Reclame Aqui e em outras plataformas de avaliação, buscando por reclamações e avaliações de clientes.
> 
> *   Investigar a existência de processos judiciais e administrativos (Procon, Vigilância Sanitária, etc.) associados ao CNPJ correto.
> 
> *   Verificar o "Edital de Abertura de Procedimento Administrativo" para determinar se a empresa está envolvida em alguma questão legal ou regulatória.
> 
> A reputação online da empresa (Reclame Aqui, avaliação de clientes), pendências legais e regulatórias são fatores cruciais para determinar a confiabilidade da Padaria Palma de Ouro.
> 
> A seguir, apresento um resumo dos pontos identificados até o momento:
> 
> | Aspecto               | Status                                                                                                                                                                                                                                 | Avaliação                                                                                                                                                   |
> | --------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------- |
> | Informações Cadastrais | Múltiplos CNPJs com nomes similares. Necessário confirmar o CNPJ correto da padaria no endereço indicado.                                                                                                                               | Em aberto                                                                                                                                                   |
> | Reputação Online      | Avaliação de 4.4 estrelas no Google Places. Necessário verificar a reputação no Reclame Aqui e outras plataformas.                                                                                                                      | Em aberto                                                                                                                                                   |
> | Pendências Legais     | Edital de Abertura de Procedimento Administrativo envolvendo empresas com nomes similares. Necessário verificar se a Padaria Palma de Ouro está relacionada. Necessário pesquisar processos judiciais e administrativos associados ao CNPJ. | Em aberto                                                                                                                                                   |
> | Website               | O site [https://palmadeouro.com.br/](https://palmadeouro.com.br/) parece ser da empresa, mas é preciso confirmar se corresponde ao CNPJ correto.                                                                                                                                                                                                                                                                |
> | Tempo de Existência   | A pesquisa indica que a "Palma de Ouro" original foi inaugurada nos anos 60.                                                                                                                                                                                                                                                                                                                              |
> 
> 


--------------------------------------------------------------

--- 👨‍⚖️ Sugestão de decisão para o consumidor ---



> ## Padaria Palma de Ouro: Análise de Confiabilidade
> 
> Considerando as informações apuradas, **NÃO RECOMENDO** realizar negócios com a Padaria Palma de Ouro neste momento. A ausência de um CNPJ validado e a existência de um edital administrativo envolvendo nomes similares levantam sérias dúvidas sobre a idoneidade da empresa.
> 
> **Resumo da Análise:**
> 
> *   **🔴 CNPJ não validado:** A falta de confirmação do CNPJ impede a verificação da regularidade fiscal e legal da empresa.
> *   **🔴 Edital Administrativo:** A existência de um edital administrativo envolvendo nomes similares exige investigação para descartar problemas legais ou regulatórios.
> *   **🟠 Reputação Online:** A avaliação de 4.4 estrelas no Google Places é um ponto positivo, mas não garante a confiabilidade da empresa. É preciso verificar a reputação em outras plataformas, como o Reclame Aqui.
> *   **🟠 Site:** O site [https://palmadeouro.com.br/](https://palmadeouro.com.br/) parece ser da empresa, mas é preciso confirmar se corresponde ao CNPJ correto.
> *   **🟢 Tempo de Existência:** A pesquisa indica que a "Palma de Ouro" original foi inaugurada nos anos 60.
> 
> **Recomendação:**
> 
> Aguarde a conclusão das investigações sobre o CNPJ, reputação online e pendências legais antes de considerar qualquer negócio com a Padaria Palma de Ouro.
> 
> **Público e Finalidade:**
> 
> Sem a validação das informações cadastrais, não é possível determinar o público-alvo ou a finalidade ideal para os produtos da Padaria Palma de Ouro.
> 
> **Alternativas:**
> 
> Enquanto aguarda a conclusão da análise, considere outras padarias na região da Bela Vista, em São Paulo, que possuam CNPJ validado e boa reputação online.


--------------------------------------------------------------
